# Détermine les trous de 10 bins ou plus dans les contacts

In [1]:
import os.path as op
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [2]:
import cooler

## Chargement des données

In [3]:
filepath = "/home/bureau/projects/def-bureau/distiller/results_iPSC/coolers_library_group/Neuron_libraries.hg38.mapq_30.2048.cool"
#filepath = "/home/bureau/projects/def-bureau/distiller/results_iPSC/coolers_library_group/Neuron_libraries.hg38.mapq_30.2048.mcool::resolutions/2048"

In [4]:
c = cooler.Cooler(filepath)

In [11]:
chr2_pixels = c.pixels(join=True).fetch("chr2")
diff_vec = np.diff(chr2_pixels["start2"][chr2_pixels["start1"]==min(chr2_pixels["start1"])])
chr2_pixels[0:len(diff_vec)][diff_vec>=20480]

,chrom1,start1,end1,chrom2,start2,end2,count
13377643,chr2,10240,12288,chr2,38912,40960,1
13377647,chr2,10240,12288,chr2,114688,116736,1
13377649,chr2,10240,12288,chr2,153600,155648,1
13377651,chr2,10240,12288,chr2,262144,264192,1
13377652,chr2,10240,12288,chr2,292864,294912,2
...,...,...,...,...,...,...,...
13377830,chr2,10240,12288,chr19,23191552,23193600,1
13377831,chr2,10240,12288,chr19,23531520,23533568,1
13377833,chr2,10240,12288,chr22,23779328,23781376,1
13377834,chr2,10240,12288,chr22,41603072,41605120,1


In [33]:
chr_pixels = c.pixels(join=True).fetch("chr9")
# Le problème, c'est de trouver une position où seul les trous sont manquants.
diff_vec = np.diff(chr_pixels["start2"][chr_pixels["start1"]==100*min(chr_pixels["start1"])])
fin_vec=np.array(chr_pixels["start2"][0:len(diff_vec)][diff_vec>=20480])
#trous = pd.DataFrame(pd.Series("chr9"),fin_vec-diff_vec[diff_vec>=20480],fin_vec,columns=["chr","start","end"])
tmp = {"chr":"chr9","start":(fin_vec-diff_vec[diff_vec>=20480]),"end":fin_vec}
trous = pd.DataFrame(tmp)
trous

,chr,start,end
0,chr9,53610496,53637120
1,chr9,85882880,85915648
2,chr9,20480,40960
3,chr9,167936,473088
4,chr9,497664,536576
...,...,...,...
60,chr9,4261888,46440448
61,chr9,34955264,58894336
62,chr9,-16146432,638976
63,chr9,-1333248,765952


In [35]:
trous.to_csv("trous_chr9.csv")

Ci-dessous j'essaie de créer une fonction pour imposer la symmétrie de la matrice de contact, car les heatmaps suggère qu'elles ne le sont pas.

In [30]:
#tmp = c.bins()[['chrom', 'start', 'end']]
#bins = tmp[0:-1]
bins = c.bins()[0:-1]
bins

,chrom,start,end,weight
0,chr1,0,2048,NaN
1,chr1,2048,4096,NaN
2,chr1,4096,6144,NaN
3,chr1,6144,8192,NaN
4,chr1,8192,10240,NaN
...,...,...,...,...
1567251,chrUn_KI270312v1,0,998,NaN
1567252,chrUn_KI270539v1,0,993,NaN
1567253,chrUn_KI270385v1,0,990,NaN
1567254,chrUn_KI270423v1,0,981,NaN


In [31]:
tri_fct = cooler.create.sanitize_pixels(bins,sort=True)

/lustre03/project/6000443/cooltools/lib/python3.8/site-packages/cooler/util.py:733: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  is_cat = pd.api.types.is_categorical(bins["chrom"])


ValueError: Lengths must match to compare